<a href="https://colab.research.google.com/github/jehoon5940/2024-Final-machine-learning-task/blob/main/Vector_Machine%2C_MNIST_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# 자동 혼합 정밀도 (최신 GPU에서만 가능)
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

# 데이터 로드 및 전처리
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 데이터 증강 적용
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)
datagen.fit(x_train)

# CNN 모델 (최적화)
model = Sequential([
    Conv2D(128, (3, 3), activation='relu', input_shape=(28, 28, 1), kernel_initializer='he_uniform'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
    BatchNormalization(),
    Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.4),

    Flatten(),
    Dense(512, activation='relu', kernel_initializer='he_uniform'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(256, activation='relu', kernel_initializer='he_uniform'),
    Dropout(0.3),

    Dense(10, activation='softmax', dtype='float32')  # mixed precision 적용
])

# 콜백 설정 (빠른 종료)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.5, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)  # patience 줄이기

# 모델 컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # 학습률 조정
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 모델 훈련 (배치 크기 증가 + steps_per_epoch 추가)
history = model.fit(datagen.flow(x_train, y_train, batch_size=128),  # 64 → 128
                    validation_data=(x_test, y_test),
                    epochs=50,  # 100 → 50
                    steps_per_epoch=len(x_train) // 128,  # 추가
                    callbacks=[lr_reduction, early_stopping])

# 모델 평가
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


468/468 ━━━━━━━━━━━━━━━━━━━━ 60s 81ms/step - accuracy: 0.5680 - loss: 1.5462 - val_accuracy: 0.9744 - val_loss: 0.0835 - learning_rate: 1.0000e-04
Epoch 2/50
  1/468 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.9062 - loss: 0.3629

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


468/468 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9062 - loss: 0.3629 - val_accuracy: 0.9749 - val_loss: 0.0828 - learning_rate: 1.0000e-04
Epoch 3/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 21s 45ms/step - accuracy: 0.9066 - loss: 0.3001 - val_accuracy: 0.9874 - val_loss: 0.0410 - learning_rate: 1.0000e-04
Epoch 4/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8828 - loss: 0.2956 - val_accuracy: 0.9873 - val_loss: 0.0411 - learning_rate: 1.0000e-04
Epoch 5/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 40s 44ms/step - accuracy: 0.9423 - loss: 0.1920 - val_accuracy: 0.9887 - val_loss: 0.0339 - learning_rate: 1.0000e-04
Epoch 6/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9688 - loss: 0.2261 - val_accuracy: 0.9888 - val_loss: 0.0338 - learning_rate: 1.0000e-04
Epoch 7/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 40s 46ms/step - accuracy: 0.9555 - loss: 0.1463 - val_accuracy: 0.9900 - val_loss: 0.0298 - learning_rate: 1.0000e-04
Epoch 8/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.968

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# 데이터 로드 및 전처리
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 데이터 증강 적용
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)
datagen.fit(x_train)

# 개선된 CNN 모델
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.4),

    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(128, activation='relu'),
    Dropout(0.3),

    Dense(10, activation='softmax')
])

# 콜백 설정
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.5, min_lr=1e-5)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 모델 훈련
history = model.fit(datagen.flow(x_train, y_train, batch_size=128),
                    validation_data=(x_test, y_test),
                    epochs=30, callbacks=[lr_reduction, early_stopping])

# 모델 평가
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Epoch 1/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 39s 61ms/step - accuracy: 0.5144 - loss: 1.6067 - val_accuracy: 0.2493 - val_loss: 2.3925 - learning_rate: 3.0000e-04
Epoch 2/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 19s 40ms/step - accuracy: 0.8848 - loss: 0.3666 - val_accuracy: 0.9853 - val_loss: 0.0457 - learning_rate: 3.0000e-04
Epoch 3/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 20s 42ms/step - accuracy: 0.9256 - loss: 0.2412 - val_accuracy: 0.9863 - val_loss: 0.0420 - learning_rate: 3.0000e-04
Epoch 4/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.9404 - loss: 0.1993 - val_accuracy: 0.9880 - val_loss: 0.0372 - learning_rate: 3.0000e-04
Epoch 5/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 19s 40ms/step - accuracy: 0.9524 - loss: 0.1576 - val_accuracy: 0.9772 - val_loss: 0.0695 - learning_rate: 3.0000e-04
Epoch 6/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 18s 38ms/step - accuracy: 0.9573 - loss: 0.1408 - val_accuracy: 0.9920 - val_loss: 0.0234 - learning_rate: 3.0000e-04
Epoch 7/30
469/469 ━━━━━━━━━━━━━━━━━━━━ 18s 38ms/ste